#  MODELOS VAR

A nivel empírico los economistas han acudido a los modelos de Vectores Autorregresivos, VAR, como herramienta básica para evaluar las políticas económicas (Galán y Venegas, 2013 y Galán, 2014). Esta metodología econométrica fue planteada inicialmente por el célebre trabajo del Nobel en Economía 2011 Christopher Sims (1980) Macroeconomic and Reality, en donde presenta una fuerte crítica hacia los modelos de sistemas ecuaciones y sus principales aplicaciones como son los modelos macroeconométricos o de gran escala.  
 
Sims menciona que la mayor parte de las restricciones que aparecen en los modelos son falsas debido a i) no hay conocimiento suficiente en la teoría económica para clasificar a las variables en endógenas y exógenas y ii) a priori no se puede establecer restricciones cero. De esta manera Sims propuso el modelo VAR, un sistema de ecuaciones autorregresivas, en donde las variables utilizadas no se distinguen si son endógenas o exógenas ya que se asume que cada una afecta y es afectada por las demás.

El modelo VAR desarrollado por Sims (1980 y 1986) ha tenido gran popularidad al ser una herramienta muy útil para el análisis empírico de las series de tiempo económicas ya que tiene las siguientes propiedades: i) parte de un enfoque ateórico, ii) es capaz de separar los efectos pasados que explican al vector de las variables endógenas a través de su pasado o mediante variables autorregresivas. 

Para ejemplificar la estimación de un modelo de Vectores Autorregresivos mediante el programa RStudio, se considera un VAR bivariado en donde se considera analizar el comportamiento de la inflación y de la oferta de dinero  para el caso de la economía mexicana tomando el periodo del primero mes del año 2000 al cuarto mes del 2014. Cabe mencionar que los datos se obtuvieron del Sistema de Información Estadística del Banco de México. 
 
Posteriormente se aplica logaritmos a las series, para ello se utiliza la base de datos que se encuentra en el archivo base_var_inflacion.csv. Con esta base se crea el siguiente objeto, mex_var, que será utilizado para la estimación del modelo VAR(p).

In [1]:
mex_var <- read.csv("base_var_inflacion.csv", header=T)
attach(mex_var)
mex_var

M2,INPC
<dbl>,<dbl>
28.7740,59.8083
29.3109,60.3388
29.8290,60.6734
30.1613,61.0186
30.4289,61.2467
30.9230,61.6095
31.6270,61.8498
31.7013,62.1896
32.0940,62.6439


Una vez que se tiene el objeto de trabajo, se procede a dar formato de series de tiempo a la base de datos  a cabo por mediante el siguiente código. Comenzando primero por el índice de la oferta monetaria y posteriormente al índice de precios. 

In [2]:
# Para M2
tm2=ts(mex_var[,1], start=2000, freq=12)

# Para INPC
tp=ts(mex_var[,2], start=2000, freq=12)

In [3]:
# Transformo estas variabes el logaritmo

ltm2 <- log(tm2)

ltp <- log(tp)


In [4]:
# grafico

ts.plot(ltp, ltm2, col=c("blue", "red"))

Del anterior gráfico se aprecia tanto el INPC como el índice M2 presenta una trayectoria determinística creciente, por lo que ambas series no satisfacen el supuesto de ruido blanco o que son estacionarias. Para corroborarlo se llevará a continuación las pruebas de raíz unitaria. Para ello se instalará la paquetería de los vectores autorregresivos, vars, posteriormente se activa la librería respectiva.

In [5]:
install.packages("vars")
library("vars")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Loading required package: MASS
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich
Loading required package: urca
Loading required package: lmtest


Para aplicar la prueba de raíz unitaria de Dickey Fuller Aumentada (ADF) se plantea la siguiente Hipótesis nula vs. Hipótesis alternativa: 
 
Ho: La variable x no tiene una raíz unitaria

Ha: La variable x tiene una raíz unitaria 

In [6]:
adf1_ltp <- summary(ur.df(ltp, lags=1))
adf1_ltp


############################################### 
# Augmented Dickey-Fuller Test Unit Root Test # 
############################################### 

Test regression none 


Call:
lm(formula = z.diff ~ z.lag.1 - 1 + z.diff.lag)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0093954 -0.0015285  0.0003127  0.0018731  0.0069307 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
z.lag.1    4.405e-04  7.598e-05   5.798 3.14e-08 ***
z.diff.lag 4.532e-01  6.741e-02   6.724 2.51e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.003061 on 172 degrees of freedom
Multiple R-squared:  0.6247,	Adjusted R-squared:  0.6204 
F-statistic: 143.2 on 2 and 172 DF,  p-value: < 2.2e-16


Value of test-statistic is: 5.798 

Critical values for test statistics: 
      1pct  5pct 10pct
tau1 -2.58 -1.95 -1.62


El valor estadístico de ADF tiene un valor de 5.798. Al ser positivo indica que INPC no es estacionario, por lo que se rechaza la hipótesis nula y se acepta la alternativa de que la variable presenta raíz unitaria.
Se continúa aplicando la prueba ADF incluyendo la constante y la tendencia respectivamente.

In [7]:
# Prueba ADF con término constante o con deriva
adf2_ltp <- summary(ur.df(ltp, type="drift", lags=12))
adf2_ltp


############################################### 
# Augmented Dickey-Fuller Test Unit Root Test # 
############################################### 

Test regression drift 


Call:
lm(formula = z.diff ~ z.lag.1 + 1 + z.diff.lag)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0068156 -0.0016026  0.0001526  0.0014280  0.0072406 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)   0.009957   0.006617   1.505 0.134478    
z.lag.1      -0.001338   0.001358  -0.985 0.326065    
z.diff.lag1   0.250487   0.074878   3.345 0.001040 ** 
z.diff.lag2  -0.113522   0.076214  -1.490 0.138467    
z.diff.lag3  -0.117347   0.076668  -1.531 0.127993    
z.diff.lag4  -0.112257   0.077012  -1.458 0.147037    
z.diff.lag5  -0.009990   0.077339  -0.129 0.897396    
z.diff.lag6  -0.259447   0.077152  -3.363 0.000981 ***
z.diff.lag7  -0.029130   0.078606  -0.371 0.711473    
z.diff.lag8  -0.077228   0.078347  -0.986 0.325867    
z.diff.lag9   0.009048   0.0

In [8]:
# Prueba ADF con tendencia
adf3_ltp <- summary(ur.df(ltp, type="trend", lags=1))
adf3_ltp


############################################### 
# Augmented Dickey-Fuller Test Unit Root Test # 
############################################### 

Test regression trend 


Call:
lm(formula = z.diff ~ z.lag.1 + 1 + tt + z.diff.lag)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.008542 -0.001558  0.000245  0.002001  0.007976 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.818e-01  9.334e-02   4.090 6.64e-05 ***
z.lag.1     -9.203e-02  2.265e-02  -4.064 7.37e-05 ***
tt           3.229e-04  8.081e-05   3.996 9.59e-05 ***
z.diff.lag   4.596e-01  6.592e-02   6.972 6.63e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.002922 on 170 degrees of freedom
Multiple R-squared:  0.2764,	Adjusted R-squared:  0.2636 
F-statistic: 21.65 on 3 and 170 DF,  p-value: 6.356e-12


Value of test-statistic is: -4.0638 18.5848 8.8833 

Critical values for test statistics: 
      1pct  5pct 10pct
tau3 

In [9]:
# Primera diferencia del logarítmo del índice de precios
dltp <- diff(ltp)
dltp

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
2000,,8.830899e-03,5.530035e-03,5.673354e-03,3.731235e-03,5.906109e-03,3.892786e-03,5.478918e-03,7.278527e-03,6.862940e-03,8.513753e-03,1.076795e-02
2001,5.528727e-03,-6.621451e-04,6.315807e-03,5.031426e-03,2.292548e-03,2.361935e-03,-2.601642e-03,5.907204e-03,9.266747e-03,4.509294e-03,3.759925e-03,1.383248e-03
2002,9.188326e-03,-6.422290e-04,5.101342e-03,5.448167e-03,2.024861e-03,4.863613e-03,2.866623e-03,3.794528e-03,5.998177e-03,4.397026e-03,8.055925e-03,4.342727e-03
2003,4.034864e-03,2.773740e-03,6.292558e-03,1.705137e-03,-3.232081e-03,8.270942e-04,1.447856e-03,2.994754e-03,5.935186e-03,3.660117e-03,8.266238e-03,4.289264e-03
2004,6.196229e-03,5.963945e-03,3.383107e-03,1.508080e-03,-2.512060e-03,1.601617e-03,2.617121e-03,6.154143e-03,8.234898e-03,6.901490e-03,8.493928e-03,2.063620e-03
2005,3.607545e-05,3.326228e-03,4.497090e-03,3.555144e-03,-2.515112e-03,-9.607781e-04,3.906345e-03,1.193606e-03,3.999251e-03,2.451791e-03,7.171486e-03,6.123418e-03
2006,5.846934e-03,1.528507e-03,1.254255e-03,1.465015e-03,-4.461013e-03,8.625786e-04,2.738955e-03,5.089289e-03,1.004434e-02,4.362644e-03,5.233555e-03,5.767272e-03
2007,5.151410e-03,2.791690e-03,2.161326e-03,-5.968665e-04,-4.890375e-03,1.200415e-03,4.238189e-03,4.065537e-03,7.734852e-03,3.889485e-03,7.030207e-03,4.125693e-03
2008,4.623879e-03,2.968366e-03,7.222161e-03,2.273235e-03,-1.081415e-03,4.129657e-03,5.557859e-03,5.757708e-03,6.792413e-03,6.790750e-03,1.130304e-02,6.901418e-03
2009,2.315167e-03,2.205139e-03,5.735818e-03,3.494204e-03,-2.917065e-03,1.839684e-03,2.720636e-03,2.389543e-03,5.003665e-03,3.020864e-03,5.174004e-03,4.130482e-03


In [10]:
# Segunda diferencia del logarítmo del indice de precios
d2ltp <- diff(dltp)
d2ltp

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
2000,,,-3.300864e-03,1.433194e-04,-1.942120e-03,2.174874e-03,-2.013323e-03,1.586132e-03,1.799609e-03,-4.155866e-04,1.650813e-03,2.254195e-03
2001,-5.239221e-03,-6.190872e-03,6.977952e-03,-1.284381e-03,-2.738878e-03,6.938669e-05,-4.963577e-03,8.508847e-03,3.359543e-03,-4.757453e-03,-7.493690e-04,-2.376678e-03
2002,7.805078e-03,-9.830555e-03,5.743571e-03,3.468257e-04,-3.423306e-03,2.838752e-03,-1.996990e-03,9.279053e-04,2.203648e-03,-1.601151e-03,3.658899e-03,-3.713197e-03
2003,-3.078636e-04,-1.261123e-03,3.518818e-03,-4.587421e-03,-4.937218e-03,4.059175e-03,6.207614e-04,1.546898e-03,2.940433e-03,-2.275070e-03,4.606121e-03,-3.976974e-03
2004,1.906965e-03,-2.322832e-04,-2.580838e-03,-1.875028e-03,-4.020139e-03,4.113677e-03,1.015504e-03,3.537022e-03,2.080755e-03,-1.333408e-03,1.592438e-03,-6.430308e-03
2005,-2.027544e-03,3.290152e-03,1.170862e-03,-9.419461e-04,-6.070256e-03,1.554334e-03,4.867123e-03,-2.712739e-03,2.805645e-03,-1.547461e-03,4.719695e-03,-1.048068e-03
2006,-2.764838e-04,-4.318427e-03,-2.742516e-04,2.107601e-04,-5.926028e-03,5.323591e-03,1.876376e-03,2.350334e-03,4.955049e-03,-5.681694e-03,8.709111e-04,5.337172e-04
2007,-6.158620e-04,-2.359720e-03,-6.303641e-04,-2.758192e-03,-4.293509e-03,6.090790e-03,3.037773e-03,-1.726515e-04,3.669315e-03,-3.845367e-03,3.140722e-03,-2.904514e-03
2008,4.981865e-04,-1.655513e-03,4.253795e-03,-4.948926e-03,-3.354650e-03,5.211072e-03,1.428201e-03,1.998493e-04,1.034705e-03,-1.663343e-06,4.512291e-03,-4.401623e-03
2009,-4.586251e-03,-1.100275e-04,3.530679e-03,-2.241614e-03,-6.411269e-03,4.756749e-03,8.809519e-04,-3.310929e-04,2.614121e-03,-1.982801e-03,2.153140e-03,-1.043522e-03


In [11]:
# Primera diferencia del logaritmo del indice de M2
dltm2 <- diff(ltm2)
dltm2

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
2000,,1.848726e-02,1.752161e-02,1.107857e-02,8.833169e-03,1.610743e-02,2.251094e-02,2.346503e-03,1.231141e-02,-2.755085e-03,1.304964e-02,7.123133e-03
2001,-1.749959e-03,2.970698e-02,1.474025e-02,1.144747e-02,5.712458e-03,1.047779e-02,1.196097e-02,2.655685e-02,1.227960e-02,1.023945e-02,2.048009e-02,8.568050e-03
2002,-8.320799e-03,1.338613e-02,1.844048e-02,7.611258e-03,6.584716e-03,1.311266e-02,9.000232e-03,6.729294e-04,4.272589e-03,6.753720e-03,1.129268e-02,2.092455e-02
2003,4.990910e-03,1.262121e-02,1.077413e-02,-1.761317e-03,1.876789e-02,6.407730e-03,1.468874e-02,-9.900348e-05,9.815108e-03,9.385220e-03,1.890639e-02,2.678762e-02
2004,-7.754015e-03,1.538607e-03,2.480115e-02,1.480451e-03,4.486596e-03,1.642854e-02,9.950161e-04,1.187579e-03,1.287950e-02,1.255595e-02,8.647242e-03,2.367688e-02
2005,6.763893e-03,7.657757e-03,2.175245e-02,5.176253e-04,1.569383e-02,1.623307e-02,5.790860e-03,8.894047e-03,1.147424e-02,1.577649e-02,1.369103e-02,7.153497e-03
2006,9.275030e-03,1.398719e-02,1.185666e-02,4.768605e-03,-8.569514e-03,1.688555e-02,2.394220e-03,-1.654651e-03,1.216839e-02,4.372547e-03,2.138119e-02,3.090347e-02
2007,-1.580543e-03,7.856195e-03,7.177317e-03,-6.020057e-03,1.563958e-02,8.922983e-03,9.928597e-03,1.029719e-02,7.054679e-03,3.137557e-03,1.696401e-02,1.312836e-02
2008,8.325621e-03,4.881002e-03,9.047516e-03,4.725984e-03,8.489755e-03,-5.363029e-03,7.184316e-03,4.146171e-03,1.657988e-02,1.570061e-02,2.190658e-02,5.655837e-02
2009,9.065925e-03,-6.960282e-03,7.377286e-03,5.121706e-03,4.371129e-04,1.375099e-03,8.583964e-03,-8.413193e-04,6.406355e-03,1.204685e-02,7.161225e-03,6.629828e-03


In [12]:
# Segunda diferencia del logaritmo del indice de M2
d2ltm2 <- diff(dltm2)
d2ltm2

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
2000,,,-0.0009656447,-0.0064430432,-0.0022454019,0.0072742598,0.0064035137,-0.0201644392,0.0099649039,-0.0150664921,0.0158047270,-0.0059265096
2001,-0.0088730913,0.0314569386,-0.0149667317,-0.0032927813,-0.0057350086,0.0047653308,0.0014831800,0.0145958761,-0.0142772414,-0.0020401552,0.0102406429,-0.0119120420
2002,-0.0168888485,0.0217069284,0.0050543500,-0.0108292217,-0.0010265422,0.0065279437,-0.0041124272,-0.0083273029,0.0035996601,0.0024811310,0.0045389594,0.0096318710
2003,-0.0159336405,0.0076303038,-0.0018470875,-0.0125354435,0.0205292106,-0.0123601637,0.0082810134,-0.0147877468,0.0099141118,-0.0004298882,0.0095211657,0.0078812381
2004,-0.0345416385,0.0092926219,0.0232625468,-0.0233207029,0.0030061446,0.0119419398,-0.0154335193,0.0001925633,0.0116919246,-0.0003235563,-0.0039087062,0.0150296349
2005,-0.0169129832,0.0008938633,0.0140946961,-0.0212348273,0.0151762071,0.0005392421,-0.0104422142,0.0031031862,0.0025801935,0.0043022503,-0.0020854570,-0.0065375363
2006,0.0021215328,0.0047121568,-0.0021305229,-0.0070880590,-0.0133381185,0.0254550678,-0.0144913343,-0.0040488707,0.0138230404,-0.0077958419,0.0170086392,0.0095222854
2007,-0.0324840149,0.0094367374,-0.0006788772,-0.0131973740,0.0216596393,-0.0067165994,0.0010056133,0.0003685911,-0.0032425089,-0.0039171221,0.0138264544,-0.0038356531
2008,-0.0048027366,-0.0034446193,0.0041665137,-0.0043215314,0.0037637710,-0.0138527844,0.0125473452,-0.0030381451,0.0124337100,-0.0008792738,0.0062059713,0.0346517958
2009,-0.0474924494,-0.0160262067,0.0143375679,-0.0022555799,-0.0046845933,0.0009379857,0.0072088651,-0.0094252829,0.0072476740,0.0056404973,-0.0048856272,-0.0005313973


El logaritmo del índice del INPC y del M2 son estacionarios mediante la segunda diferencia.

Graficando las series las segundas diferencias del logaritmo del índice de precios, d2ltp, y el logaritmo del índice de la oferta de dinero, d2ltm2, mediante el siguiente código se obtiene la Gráfica 2

In [13]:
ts.plot(d2ltp, d2ltm2, col=c("blue", "red"))

In [14]:
library(lmtest)

A continuación se lleva a cabo las pruebas de causalidad en el sentido de Granger para determinar el orden causal entre las variables, para ello se activa la librería lmtest. Una vez activa la librería, se aplica de la siguiente manera: i) Se verifica la dirección de la causalidad de la oferta de dinero hacia los precios, donde la hipótesis nula implica que la oferta de dinero no causa en el sentido de Granger a los precios:

In [15]:
grangertest(d2ltp ~ d2ltm2, order = 1)

Res.Df,Df,F,Pr(>F)
<dbl>,<dbl>,<dbl>,<dbl>
170,NA,NA,NA
171,-1,0.8795894,0.3496447


Se acepta la hipótesis nula que la oferta de dinero no causa en el sentido de Granger a los precios, dado que el p-value, 0.3496, es mayor al valor crítico del 5 por ciento.

Se verifica la dirección de la  causalidad de los precios hacia la oferta de dinero, donde la hipótesis nula implica que los precios no causan en el sentido de Granger  a la oferta de dinero.

In [16]:
grangertest(d2ltm2 ~ d2ltp, order = 1)

Res.Df,Df,F,Pr(>F)
<dbl>,<dbl>,<dbl>,<dbl>
170,NA,NA,NA
171,-1,0.09088673,0.7634213


Se acepta la hipótesis nula que los precios no causa en el sentido de Granger la oferta de dinero, dado que el p-value 0.7634, es mayot al valor crítico del 5%.

Se Identifica el VAR, a través de los criterios de información:

Akaike (AIC), Hanna-Quin (HQ), Shwarz (SC), y Error de Predicción Final (FPE).

In [17]:
mex_var2<-data.frame(d2ltm2,d2ltp)

In [18]:
VARselect(mex_var2, lag.max=12)

,1,2,3,4,5,6,7,8,9,10,11,12
AIC(n),-2.046463e+01,-2.068196e+01,-2.065764e+01,-2.075149e+01,-2.075348e+01,-2.090388e+01,-2.097478e+01,-2.104458e+01,-2.103653e+01,-2.109132e+01,-2.130916e+01,-2.130390e+01
HQ(n),-2.041820e+01,-2.060458e+01,-2.054931e+01,-2.061220e+01,-2.058323e+01,-2.070268e+01,-2.074263e+01,-2.078147e+01,-2.074247e+01,-2.076631e+01,-2.095320e+01,-2.091699e+01
SC(n),-2.035027e+01,-2.049137e+01,-2.039081e+01,-2.040842e+01,-2.033417e+01,-2.040834e+01,-2.040300e+01,-2.039656e+01,-2.031228e+01,-2.029083e+01,-2.043244e+01,-2.035094e+01
FPE(n),1.295173e-09,1.042211e-09,1.067940e-09,9.723977e-10,9.706467e-10,8.353390e-10,7.784502e-10,7.263246e-10,7.326459e-10,6.941160e-10,5.587598e-10,5.623294e-10


Una vez identificado el VAR se procede con el siguiente código su estimación, VAR. En Tabla 7 se presentan las ecuaciones del VAR de orden 11, VAR(11).

In [19]:
var1 <- VAR(mex_var2, p=11)
var1


VAR Estimation Results:

Estimated coefficients for equation d2ltm2: 
Call:
d2ltm2 = d2ltm2.l1 + d2ltp.l1 + d2ltm2.l2 + d2ltp.l2 + d2ltm2.l3 + d2ltp.l3 + d2ltm2.l4 + d2ltp.l4 + d2ltm2.l5 + d2ltp.l5 + d2ltm2.l6 + d2ltp.l6 + d2ltm2.l7 + d2ltp.l7 + d2ltm2.l8 + d2ltp.l8 + d2ltm2.l9 + d2ltp.l9 + d2ltm2.l10 + d2ltp.l10 + d2ltm2.l11 + d2ltp.l11 + const 

    d2ltm2.l1      d2ltp.l1     d2ltm2.l2      d2ltp.l2     d2ltm2.l3 
-7.629557e-01 -1.714432e-01 -8.434147e-01  1.109554e-01 -6.063436e-01 
     d2ltp.l3     d2ltm2.l4      d2ltp.l4     d2ltm2.l5      d2ltp.l5 
-6.402061e-03 -6.978840e-01 -6.657233e-02 -6.033920e-01 -1.467567e-01 
    d2ltm2.l6      d2ltp.l6     d2ltm2.l7      d2ltp.l7     d2ltm2.l8 
-5.911775e-01  1.071395e-01 -4.820850e-01 -5.209120e-01 -4.042083e-01 
     d2ltp.l8     d2ltm2.l9      d2ltp.l9    d2ltm2.l10     d2ltp.l10 
 5.843583e-02 -2.957712e-01 -2.264201e-01 -2.607288e-01 -3.519280e-01 
   d2ltm2.l11     d2ltp.l11         const 
-1.952368e-01 -4.569036e-02 -1.772883e

In [20]:
# Me fijo si el VAR estimado satisface la condiión de
# estabilidad, utilizando el código summary para que R muestre 
# las raíces del polinomio característico así como los 
# estadísticos necesarios para llevar a cabo la inferencia
# estadística.
summary(var1)


VAR Estimation Results:
Endogenous variables: d2ltm2, d2ltp 
Deterministic variables: const 
Sample size: 163 
Log Likelihood: 1316.416 
Roots of the characteristic polynomial:
0.9851 0.9851 0.9604 0.9604 0.9425 0.9425 0.9343 0.9343 0.906 0.906 0.8852 0.8852 0.8749 0.8749 0.8717 0.8717 0.8679 0.8553 0.8553 0.8127 0.8127 0.7702
Call:
VAR(y = mex_var2, p = 11)


Estimation results for equation d2ltm2: 
d2ltm2 = d2ltm2.l1 + d2ltp.l1 + d2ltm2.l2 + d2ltp.l2 + d2ltm2.l3 + d2ltp.l3 + d2ltm2.l4 + d2ltp.l4 + d2ltm2.l5 + d2ltp.l5 + d2ltm2.l6 + d2ltp.l6 + d2ltm2.l7 + d2ltp.l7 + d2ltm2.l8 + d2ltp.l8 + d2ltm2.l9 + d2ltp.l9 + d2ltm2.l10 + d2ltp.l10 + d2ltm2.l11 + d2ltp.l11 + const 

             Estimate Std. Error t value Pr(>|t|)    
d2ltm2.l1  -7.630e-01  8.274e-02  -9.221 4.04e-16 ***
d2ltp.l1   -1.714e-01  2.356e-01  -0.728 0.468104    
d2ltm2.l2  -8.434e-01  1.012e-01  -8.333 6.58e-14 ***
d2ltp.l2    1.110e-01  2.530e-01   0.438 0.661704    
d2ltm2.l3  -6.063e-01  1.216e-01  -4.988 1.78e-06 *

In [21]:
# Grafico la variable observada vs la estimada
plot(var1)

In [22]:
# Se hacen las pruebas de especificación.
# Se verifica la existencia de autocorrelación serial en los
# residuos
seriala <- serial.test(var1, lags.pt=11, type="PT.asymptotic")
seriala$serial


	Portmanteau Test (asymptotic)

data:  Residuals of VAR object var1
Chi-squared = 27.28, df = 0, p-value < 2.2e-16


Se rechaza la hipótesis nula de que los residuales no están correlacionados y se acepta la hipótesis alterna de que hay prescencia de correlación serial entre los residuales. 
Debo verificar si los residuales del modelo VAR estimado se distribuyen como una normal.

In [23]:
normalidad <- normality.test(var1)
normalidad$jb.mul

$JB

	JB-Test (multivariate)

data:  Residuals of VAR object var1
Chi-squared = 201.07, df = 4, p-value < 2.2e-16


$Skewness

	Skewness only (multivariate)

data:  Residuals of VAR object var1
Chi-squared = 25.209, df = 2, p-value = 3.357e-06


$Kurtosis

	Kurtosis only (multivariate)

data:  Residuals of VAR object var1
Chi-squared = 175.86, df = 2, p-value < 2.2e-16



Los residuales no se distribuyen de forma normal. Presentan problemas en la curtosis y en el sesgo. Se verifica si a varianza de los residuales son o no homocedasticos.

In [24]:
arch1 <- arch.test(var1, lags.multi=11)
arch1$arch.mul


	ARCH (multivariate)

data:  Residuals of VAR object var1
Chi-squared = 80.177, df = 99, p-value = 0.917


A través de la prueba de Heteroscedasticidad se tiene que los residuales si satisfacen el supuesto de varianza constante.  El modelo VAR(11) estimado tiene problemas de especificación de autocorrelación y de normalidad. Observando las Gráficas 3a y 3b se puede encontrar que existe un punto de ruptura o de cambio estructural, esto implica que se puede introducir una variable dummy para ajustar el modelo, en donde los valores de uno indiquen que en ese periodo existe un cambio estructural, mientras los valores cero se refiere la ausencia de cambio estructural. 
 
Una vez que se han llevado a cabo las pruebas de especificación y verificar que el modelo VAR las satisface, en su caso corregirlo, se procede a realizar el análisis impulso respuesta permitiendo con ello observar la trayectoria de la variable de estudio. 

In [26]:
# Análisis impulso respuesta permitiendo con ello observar la
# trayectoria de la variable de estudio
var1_irflp <- irf(var1, response="d2ltp", n.ahead=8, boot=TRUE)
var1_irflp


Impulse response coefficients
$d2ltm2
              d2ltp
 [1,]  9.794980e-05
 [2,] -1.816909e-04
 [3,]  3.202127e-04
 [4,] -3.024661e-05
 [5,]  2.783473e-04
 [6,] -7.639939e-04
 [7,]  1.452209e-04
 [8,]  2.217182e-05
 [9,] -1.044960e-04

$d2ltp
              d2ltp
 [1,]  2.605455e-03
 [2,] -1.503055e-03
 [3,] -8.185969e-04
 [4,] -3.260213e-04
 [5,]  3.965302e-05
 [6,]  1.615598e-04
 [7,] -4.361451e-04
 [8,]  1.952075e-04
 [9,]  1.524599e-04


Lower Band, CI= 0.95 
$d2ltm2
              d2ltp
 [1,] -0.0003204340
 [2,] -0.0005506004
 [3,] -0.0001406069
 [4,] -0.0004759758
 [5,] -0.0002520763
 [6,] -0.0011382156
 [7,] -0.0002222396
 [8,] -0.0003530651
 [9,] -0.0005109387

$d2ltp
              d2ltp
 [1,]  0.0021099520
 [2,] -0.0017321572
 [3,] -0.0011406685
 [4,] -0.0007444765
 [5,] -0.0003010009
 [6,] -0.0002365564
 [7,] -0.0007457807
 [8,] -0.0002658217
 [9,] -0.0002689366


Upper Band, CI= 0.95 
$d2ltm2
              d2ltp
 [1,]  0.0004099953
 [2,]  0.0001829124
 [3,]  0.0007333412
 

In [27]:
# Grafico el impulso respuesta
plot(var1_irflp)

In [28]:
var1_irflm2 <- irf(var1, response="d2ltm2", n.ahead=5, boot=TRUE)
var1_irflm2


Impulse response coefficients
$d2ltm2
            d2ltm2
[1,]  0.0081332754
[2,] -0.0062221221
[3,] -0.0020705022
[4,]  0.0018202862
[5,] -0.0015104879
[6,]  0.0002519752

$d2ltp
            d2ltm2
[1,]  0.000000e+00
[2,] -4.466874e-04
[3,]  8.875805e-04
[4,] -3.435515e-04
[5,] -4.144000e-04
[6,]  5.944505e-05


Lower Band, CI= 0.95 
$d2ltm2
            d2ltm2
[1,]  6.305182e-03
[2,] -7.633627e-03
[3,] -3.453678e-03
[4,] -1.095618e-05
[5,] -2.753612e-03
[6,] -9.613176e-04

$d2ltp
            d2ltm2
[1,]  0.0000000000
[2,] -0.0014240252
[3,] -0.0006238313
[4,] -0.0016570790
[5,] -0.0015729467
[6,] -0.0013145617


Upper Band, CI= 0.95 
$d2ltm2
            d2ltm2
[1,]  9.058913e-03
[2,] -4.017654e-03
[3,] -8.005846e-05
[4,]  3.315620e-03
[5,]  4.050722e-04
[6,]  1.562514e-03

$d2ltp
           d2ltm2
[1,] 0.0000000000
[2,] 0.0005818750
[3,] 0.0020050368
[4,] 0.0012227704
[5,] 0.0008423688
[6,] 0.0014470172


In [29]:
plot(var1_irflm2)

In [30]:
# Se obtiene el análisis de la descomposición de varianza
var1_fevd_d2lp <- fevd(var1, n.ahead=50)$d2ltp
var1_fevd_d2lp

d2ltm2,d2ltp
0.001411324,0.9985887
0.004687011,0.9953130
0.014716082,0.9852839
0.014649605,0.9853504
0.022244258,0.9777557
0.075732418,0.9242676
0.076200301,0.9237997
0.075975752,0.9240242
0.076736410,0.9232636
0.092811244,0.9071888


In [31]:
var1_fevd_d2lm2 <- fevd(var1, n.ahead=50)$d2ltm2
var1_fevd_d2lm2

d2ltm2,d2ltp
1.0000000,0.000000000
0.9981009,0.001899115
0.9910356,0.008964365
0.9902672,0.009732756
0.9889929,0.011007060
0.9889689,0.011031145
0.9849030,0.015097016
0.9626215,0.037378536
0.9418736,0.058126436
0.9405360,0.059464012
